In [5]:
import time

In [6]:
def genFunc():
    i = 0
    while(i < 1000000):
        time.sleep(1)
        yield i
        i += 1

In [13]:
genFunc_instance = genFunc()

In [14]:
arr = []
while(True):
    arr.append(genFunc_instance.__next__())

KeyboardInterrupt: 

In [16]:
len(arr)

7

In [2]:
genFunc_instance.__next__()

NameError: name 'genFunc_instance' is not defined

In [18]:
import numpy as np

In [6]:
with open('logs/Testpool.log', 'r') as f:
    results = []
    for line in f:
        line = line.strip()
        results.append(line)

In [14]:
newResults = []
for result in results:
    result = result.replace('(', '')
    result = result.replace(')', '')
    result = result.split(',')
    t = []
    for k in result:
        t.append(int(k))
    newResults.append((t[0], t[1], t[2], t[3]))

In [19]:
recalls = []
recalls_of_word = []
precisions = []
precisions_of_words = []
for entry in newResults:
    (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
    recalls.append(lemma_match/n_dcsWords)
    recalls_of_word.append(word_match/n_dcsWords)

    precisions.append(lemma_match/n_output_nodes)
    precisions_of_words.append(word_match/n_output_nodes)
print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))

Avg. Micro Recall of Lemmas: 0.8274110011837282
Avg. Micro Recall of Words: 0.687829315358227
Avg. Micro Precision of Lemmas: 0.7674873412040732
Avg. Micro Precision of Words: 0.6388866681193359


#### CSV of feature codes

In [36]:
import os
from collections import defaultdict
import csv

In [9]:
for f in os.listdir('outputs/'):
    if 'featureCode' in f:
        print(f)

top_1L_bigram_featureCode.txt
top_4K_bigram_mir_featureCode.txt
top_rfe_4K_bigram_featureCode.txt
top_400_featureCode.txt
top_1L_pmi_rfe_featureCode.txt
top_4K_pmi_rfe_featureCode.txt
top_1L_pmi_featureCode.txt
top_1L_bigram_rfe_featureCode.txt
top_4K_pmi_featureCode.txt
top_1L_pmi_mir_featureCode.txt
top_1L_bigram_featureCode_d2k.txt
top_4K_pmi_mir_featureCode.txt
top_rfe_4K_bigram_featureCode_d2k.txt


In [19]:
codeOrder = {}
codeOrder['p3_1L_bigram_mir'] =  0
codeOrder['p3_1L_bigram_rfe'] =  1
codeOrder['p3_1L_pmi_mir'] =  2
codeOrder['p3_1L_pmi_rfe'] =  3

codeOrder['p2_4K_bigram_mir'] =  4
codeOrder['p2_4K_bigram_rfe'] =  5
codeOrder['p2_4K_pmi_mir'] =  6
codeOrder['p2_4K_pmi_rfe'] =7

codeFiles = {}

codeFiles['p3_1L_bigram_mir'] = 'top_1L_bigram_featureCode.txt'
codeFiles['p3_1L_bigram_rfe'] = 'top_1L_bigram_rfe_featureCode.txt'
codeFiles['p3_1L_pmi_mir'] = 'top_1L_pmi_mir_featureCode.txt'
codeFiles['p3_1L_pmi_rfe'] = 'top_1L_pmi_rfe_featureCode.txt'

codeFiles['p2_4K_bigram_mir'] = 'top_4K_bigram_mir_featureCode.txt'
codeFiles['p2_4K_bigram_rfe'] = 'top_rfe_4K_bigram_featureCode.txt'
codeFiles['p2_4K_pmi_mir'] = 'top_4K_pmi_mir_featureCode.txt'
codeFiles['p2_4K_pmi_rfe'] = 'top_4K_pmi_rfe_featureCode.txt'

In [31]:
featureLog = defaultdict(lambda: [0]*8)

In [33]:
for code in codeFiles.keys():
    fcf = codeFiles[code]
    with open(os.path.join('outputs/', fcf)) as fh:
        for _ in range(1500):
            line = next(fh)
            k = line.strip().replace(',', '*')
            featureLog[k][codeOrder[code]] = 1

In [45]:
with open('outputs/featureStats.csv', 'w') as fh:
    fs_csv = csv.writer(fh)
    hr = list(codeOrder.keys())
    hr.sort(key = lambda x: codeOrder[x])
    hr2 = ['FeatureName']
    hr2.extend(hr)
    fs_csv.writerow(hr2)
with open('outputs/featureStats.csv', 'a') as fh:
    fs_csv = csv.writer(fh)
    for key, item in featureLog.items():
        r = [key]
        r.extend(item)
        fs_csv.writerow(r)

In [46]:
hr2

['FeatureName',
 'p3_1L_bigram_mir',
 'p3_1L_bigram_rfe',
 'p3_1L_pmi_mir',
 'p3_1L_pmi_rfe',
 'p2_4K_bigram_mir',
 'p2_4K_bigram_rfe',
 'p2_4K_pmi_mir',
 'p2_4K_pmi_rfe']

#### Lemma 0/1 csv

In [1]:
import csv, os, pickle
import bz2

In [2]:
def open_dsbz2(filename):
    with bz2.BZ2File(filename, 'r') as f:
        loader = pickle.load(f)
    
    conflicts_Dict_correct = loader['conflicts_Dict_correct']
    nodelist_to_correct_mapping = loader['nodelist_to_correct_mapping']
    nodelist_correct = loader['nodelist_correct']
    featVMat_correct = loader['featVMat_correct']
    featVMat = loader['featVMat']
    conflicts_Dict = loader['conflicts_Dict']
    nodelist = loader['nodelist']
    
    return (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat)

In [2]:
files = os.listdir('outputs/dump_predictions/')

In [3]:
files

['pred_1L_bigram_proc4.csv',
 'lemma_label_BM3.csv',
 'BM3_NLoss_proc1.csv',
 'BM3_NLoss_proc0.csv',
 'pred_1L_bigram_proc5.csv',
 'BM3_NLoss_proc3.csv',
 'BM3_NLoss_proc2.csv',
 'pred_1L_bigram_proc3.csv',
 'pred_1L_bigram_proc2.csv',
 'pred_1L_bigram_proc1.csv',
 'pred_1L_bigram_proc0.csv',
 'lemmawise_labelled.csv']

In [6]:
with open('outputs/dump_predictions/lemmawise_labelled.csv', 'w') as out_fh:
    out_fh_csv = csv.writer(out_fh)
    fi = 0
    for root_file in files:
        with open(os.path.join('outputs/dump_predictions/', root_file)) as fh:
            print('Processing File: ', root_file)
            fh_csv = csv.reader(fh)
            for lr in fh_csv:
                if fi % 100 == 0:
                    print('Files done: ', fi)
                fi += 1
                sent_id = lr[0]
                dcs_name = sent_id + '.ds.bz2'
                (nodelist_correct, _, _, nodelist_to_correct_mapping,\
                    _, _, _) = open_dsbz2(os.path.join('../NewData/skt_dcs_DS.bz2_1L_bigram_heldout/', dcs_name))
                for rx in range(5):
                    lr = next(fh_csv)[1:]
                    if rx == 3:
                        iam = [int(x) for x in lr]
                for i in range(len(nodelist_correct)):
                    out_fh_csv.writerow([sent_id, nodelist_correct[i].lemma, 1*(nodelist_to_correct_mapping[i] in iam)])

Processing File:  pred_1L_bigram_proc4.csv
Files done:  0
Files done:  100
Files done:  200
Files done:  300
Files done:  400
Files done:  500
Files done:  600


KeyboardInterrupt: 

247483 avIci 0
247483 naraka 1
247483 yatra 1
247483 yatra 0
247483 yAmala 1
247483 parvata 1


In [34]:
nodelist_to_correct_mapping

{0: 1, 1: 3, 2: 4, 3: 6, 4: 8, 5: 17}